In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,30191
2,Huelva,Confirmados PDIA 14 días,1305
3,Huelva,Tasa PDIA 14 días,"254,30169339595065"
4,Huelva,Confirmados PDIA 7 días,646
5,Huelva,Total Confirmados,30370
6,Huelva,Curados,27546
7,Huelva,Fallecidos,376


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  30191.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7893.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 851 personas en los últimos 7 días 

Un positivo PCR cada 382 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,30191.0,646.0,1305.0,513170.0,125.884210,254.301693,262.0
Condado-Campiña,9751.0,308.0,597.0,156231.0,197.143973,382.126467,130.0
Huelva-Costa,17427.0,283.0,606.0,289548.0,97.738544,209.291724,111.0
Huelva (capital),7893.0,169.0,377.0,143837.0,117.494108,262.102241,68.0
Almonte,1727.0,73.0,155.0,24507.0,297.874077,632.472355,31.0
Moguer,1215.0,59.0,99.0,21867.0,269.812960,452.737001,27.0
Sierra de Huelva-Andévalo Central,2669.0,52.0,99.0,67391.0,77.161639,146.903889,18.0
Bollullos Par del Condado,943.0,46.0,93.0,14387.0,319.733092,646.416904,17.0
Palma del Condado (La),1113.0,47.0,70.0,10801.0,435.144894,648.088140,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Galaroza,41.0,2.0,11.0,1382.0,144.717800,795.947902,0.0
Villarrasa,163.0,13.0,16.0,2211.0,587.969245,723.654455,10.0
Palma del Condado (La),1113.0,47.0,70.0,10801.0,435.144894,648.088140,15.0
Bollullos Par del Condado,943.0,46.0,93.0,14387.0,319.733092,646.416904,17.0
Almonte,1727.0,73.0,155.0,24507.0,297.874077,632.472355,31.0
Cabezas Rubias,36.0,4.0,4.0,706.0,566.572238,566.572238,2.0
Moguer,1215.0,59.0,99.0,21867.0,269.812960,452.737001,27.0
Manzanilla,94.0,5.0,9.0,2118.0,236.071766,424.929178,4.0
Punta Umbría,781.0,26.0,64.0,15355.0,169.325952,416.802345,11.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Galaroza,41.0,2.0,11.0,1382.0,144.717800,795.947902,0.0,0.181818
Villalba del Alcor,480.0,2.0,9.0,3366.0,59.417706,267.379679,0.0,0.222222
Rociana del Condado,538.0,5.0,17.0,7939.0,62.980224,214.132762,3.0,0.294118
Lucena del Puerto,243.0,3.0,10.0,3261.0,91.996320,306.654400,2.0,0.300000
Nerva,173.0,4.0,13.0,5169.0,77.384407,251.499323,3.0,0.307692
San Juan del Puerto,475.0,7.0,22.0,9411.0,74.381043,233.768994,4.0,0.318182
Lepe,2106.0,7.0,22.0,27880.0,25.107604,78.909613,1.0,0.318182
Beas,204.0,1.0,3.0,4341.0,23.036167,69.108500,0.0,0.333333
Gibraleón,606.0,6.0,15.0,12737.0,47.106854,117.767135,1.0,0.400000
